# t-SNE

## Abstract
**To be done**

## 1. Background
In practice, high-dimensional data is often concentrated near a lower dimensional manifold. This can occur for a variety of reasons, but is commonly viewed as a signal that the underlying data-generating mechanism depends on significantly fewer parameters than the dimension of the observed data. Consider image data summarizing a single individual's face at a random distance and angle: as an array, each image could be extremely high dimensional, despite most of the variability depending on only two parameters. 

Recovering the underlying manifold structure is desirable in some applications and necessary in others. Data visualization is fundamentally easier in lower dimensional spaces; more accurate visualizations allow an analyst to exploit patterns in the data during modelling and inference. Computationally, low-dimensional data takes less memory to store, and is always faster to process or summarize with standard algorithms. Mathematically, low-dimensional data is unafflicted by the curse of dimensionality. For all of these reasons, the results of low-dimensional inference or modelling are generally more accurate and informative than inference performed on the original data. For example, consider analyzing images of a baseball player mid-throw. Each image could be viewed as an array in some larger space, say $\mathbb{R}^{N\times N}$. Interpolating between two such images in Euclidean space (e.g. drawing a line between the two arrays) would produce nonsensical, physically imposible results. One must learn the underlying image manifold, and then interpolate in that space.

Formally, learning an underlying manifold is necessary when the true data generating mechanism involves sampling points $y_i\in \mathbb{R}^d$, $i=1,...,n$ from some manifold $M\subseteq \mathbb{R}^d$, and then embedding these points into ambient space via a function $f:M\to \mathbb{R}^D$, $D>d$. The analyst observes $x_i = f(y_i)$ and needs to construct estimates $\widehat y_i$ of $y_i$. Different methods of manifold learning place different assumptions on $f$. For example, principal component analysis (PCA) assumes $f$ is linear, constructs the $d$ dimensional plane $M$ that explains most of the variability in the observed data, and takes $\widehat y_i$ to be the projection of $x_i$ onto that plane. PCA is fast, but fails to handle nonlinear embeddings. An alternative is Isomap, which assumes $f$ is an isometry (i.e. distance preserving), and thus tries to choose $\widehat y_i$ such that the distance between $\widehat y_i$ and $\widehat y_j$ is close to the distance between $x_i$ and $x_j$. Intuitively, points that are far apart in the ambient space should be sent to points that are far apart in the low-dimensional space, and points that are close together should stay close together. The key step in Isomap is to build a graph by defining an edge $x_ix_j$ with weight $||x_i-x_j||$ if $||x_i-x_j||$ is small (and no edge if this is large). Distances between distance points $x_k$, $x_\ell$ are estimated as the shortest path along this graph, and then multidimensional scaling is used to find a linear embedding which preserves these distances. Isomap is still quite fast, but fails when $f$ is not an isometry (e.g. when $f$ is only conformal). Furthermore, Isomap is only guaranteed to produce reliable estimates when the true $y_i$ are sampled uniformly from $M$.

Symmetric stochastic neighbor embedding (S-SNE) is another approach to nonlinear manifold learning which instead attempts to preserve a notion of probabilistic similarities between points. In the ambient space the data are assumed to be jointly Gaussain; a similarity matrix is constructed using the conditional probability that $x_j$ came from a Gaussian centered as $x_i$. S-SNE searches for low-dimensional $\widehat y_i$ that also come from a Gaussian distribution and preserve the similarity matrix from the ambient space, where "preserve" is measured using the Kullback–Leibler (KL) divergence. S-SNE struggles when one point has many neighbours all at a similar distance: for example, assume $x_1$ is at the center of a cirle and all $x_j$, $j>1$ live on this circle. Embedding the $x_i$ into one dimension would force many (very different) $x_j$ onto the same points. Naive solutions such as modifying the modifying the KL-divergence penalty result in horribly slow optimization problems. In this document we implement and study t-distributed stochastic neighbor embedding (t-SNE), which replaces the Gaussian assumption on $y_i$ with a t-distribution assumption. A heavier tailed distribution forces points at moderate distances in the ambient space to be embedded very far apart, and thus ameliorates crowding.


In particular, we implement t-SNE in basic python, and then significantly improve our implementation using numpy broadcasting, just-in-time (JIT) numba C++ compilation, and parallelization. Each version of our implementation is thoroughly benchmarked. The fastest versions of our implementation are then tested on simualted data, the MNIST digits dataset, and <font color=red>**some other data**</font> Finally, we compare our implementation to PCA and Isomap on difficult nonlinear simulations as well as the real datasets previously mentioned.

## 2. Description of Algorithm

### 2.1. Preliminaries
The algorithm begins by converting high-dimensional data $x_1,...,x_n$ into an $n\times n$ matrix of conditional probabilities. These conditional probabilities measure the similarity between points under a pre-specified Gaussian distribution. More formally, consider a multivariate Gaussian distribution centered at a single point $x_i$ with isotropic variance $\sigma_i^2$; let $f_i$ be the probability density function for this distribution. The conditional probability $p_{j|i}$ is the probability that $x_j$ would pick $x_i$ as its neighbour (out of all $\{x_1,...,x_n\}\setminus \{x_j\}$ if neighbours were chosen randomly with probabilities $f_i(x_j)$. In symbols:
$$
p_{j|i} = \frac{\exp(-||x_i-x_j||^2/2\sigma_i^2)}{\sum_{k\neq i}\exp(-||x_i-x_k||^2/2\sigma_i^2)}.
$$
Conditional probabilities are converted to joint probabilities via
$$
p_{ij} = \frac{p_{j|i}+p_{i|j}}{2n}.
$$
If $x_i$ and $x_j$ are close together from the perspective of a Gaussian distribution, then $p_{ij}$ will be large. If they are far apart, then $p_{ij}$ will be small. In this sense $\sigma_i^2$ controls what "close together" means; these variances are chosen so that the entropy equals a user specified value called the perplexity:
$$
\text{Perp}(\sigma_1,...,\sigma_n) = 2^{-\sum_j p_{j|i}\log_2p_{i|j}}.
$$
Low-dimensional points $y_1,...,y_d$ are modelled with a $t$ distribution, wherein the joint probabilities are
$$
q_{ij} = \frac{(1+||y_i-y_j||)^{-1}}{\sum_{k\neq \ell}(1+||y_k-y_\ell||)^{-1}}.
$$
Intuitively, we want the distribution of the $y_i$ to "look similar" to the distribution of the $x_i$. The KL-divergence measures similarity between probability distributions; as such t-SNE aims to minimize
$$
KL(P||Q) =\sum_{i, j}p_{ij}\log\bigg(\frac{p_{ij}}{q_{ij}}\bigg).
$$
One can compute the gradient directly as
$$
\frac{dKL(P||Q)}{dy_i} = 4\sum_{j}\frac{p_{ij}-q_{ij}}{(1+||y_i-y_j||)^{-1}}(y_i-y_j).
$$

### 2.2. The Algorithm

We now have everything we need to state the main algorithm.

---
**Algorithm 1:** t-SNE  
**Inputs:**  
High dimensional data - $X = \{x_1,...,x_n\}$,  
Desired perplexity - $Perp$,  
Number of iterations - $T$,  
Learning rate - $\eta$,  
Momentum - $\alpha(t)$  
**Outputs:**  
Low-dimensional latent factors - $Y = \{y_1,...,y_n\}$.

**Begin:**
1. Optimize $\sigma_i$ to the desired perplexity $Perp$ (e.g. with binary search).
2. Compute $p_{ij}$ using $X$.
3. Sample initial solution $Y^{(0)}\sim N(0, 10^{-4})$ entrywise.
4. For $t=0,...,T$:
  1. Compute $q_{ij}$ using $Y^{(0)}$.
  2. Compute $dKL(P||Q)/dY$ using $Y^{(0)}$.
  3. Set $Y^{(t)}=Y^{(t-1)}+\eta dKL(P||Q)/dY - \alpha(t)(Y^{(t-1)}-Y^{(t-2)})$  
  
**End**

---

For completeness, we also outline binary search.

---
**Algorithm 2:** Binary search  
**Inputs:**  
Evaluation function - $f$,  
Target value - $target$,  
Tolerance - $tol$,  
Number of iterations - $T$,  
Lower bound - $LB$,  
Upper bound - $UB$,  
**Outputs:**  
Estimated solution $x$ to $f(x)=target$.

**Begin:**
1. For $t=1,...,T$ or until converged:
  1. Set $guess = (LB+UB)/2$.  
  2. Compute $val=f(guess)$.
  3. If $val > target$, set $upper=guess$. Otherwise set $lower=guess$.
  
**End**

---


### 2.3. Accessories
Overflow is a concern when computing $||x_i-x_j||$ in high dimensions. As such, we augment our implementation to run PCA whenever the dimension of the input data is larger than $30$. t-SNE is run on the resulting $30$-dimensional linear factors.

The above algorithm offers significant gains over S-SNE, but can nonetheless be improved with a small number of tricks. The first trick is known as exaggeration: for the first several (say $50$) iterations of t-SNE, we use $4p_{ij}$ instead of $p_{ij}$. The $q_{ij}$, which always sum to $1$, will never be able to adequately model these exaggerated joint probabilities. Consequently, the cost function forces all of the $q_{ij}$ to model the largest probabilities; this has the effect of forming tight widely seperated clusters early on, which makes it easier for the clusters to move around eachother once the exaggeration has been removed.

Another trick is to use a learning rate schedule, $\eta(t)$, instead of a constant $\eta$. Adaptive learning rates accelerate training in the early stages of the algorithm (which compounds the effects of exaggeration), as well as reduce the risk of choosing an inappropriate learning rate for the application at hand. Our implementation of t-SNE allows for custom learning rate functions (as long as $\eta_t$ depends only on $t$, $\eta_{t-1}$, $\eta_0$, and constants), but also comes with the following default schedules:
$$
\begin{align*}
&\text{Constant:}\quad \eta_{t+1} = c, \quad c>0, \\
&\text{Time-based:}\quad \eta_{t+1} = \frac{\eta_t}{1+dt}, \quad d>0, \\
&\text{Step-based:}\quad \eta_{t+1} = \eta_0d^{\text{floor}(1+t)/r}, \quad d>0, r>0, \\
&\text{Exponential :}\quad \eta_{t+1} = \eta_0\exp(-dn), \quad d>0.
\end{align*}
$$
<font color=red>**These parameters control how the learning rate decays with time. For example, a step-based decay with $d=1/3$ and $r=10$ corresponds to dividing the learning rate by $3$ every $10$ iterations.**</font> We choose a small momentum term for the first few iterations (e.g. $\alpha(t)=0.5$ for $t<250$). Once the data are reasonably well seperated, increasing the momentum allows us to speed toward an extreme point (e.g. $\alpha(t)=0.2$ for $t\geq 250$).

<font color=red>**Everything below here is new.**</font> Finally, our implementation allows one to modify the distribution of the embedded data to incorporate heavier tails. The original implementation of t-SNE uses a t-distribution with one degree of freedom to model the embedded $y_i$; Kobak et al. [3] show that replacing the standard t-distribution kernel with $\nu$ degrees of freedom
$$
p(d|\nu) = \frac{1}{(1+d^2/\nu)^{(\nu+1)/2}}
$$
with the scaled kernel 
$$
p(d|\lambda) = \frac{1}{(1+d^2/\lambda)^{\lambda}}
$$
also yields a simple, closed-form update for the gradient - namely
$$
\frac{dKL(P||Q)}{dy_i} = 4\sum_{j}\frac{p_{ij}-q_{ij}}{(1+||y_i-y_j||/\lambda)^{-1}}(y_i-y_j).
$$
As $\lambda \to \infty$, the embedded distribution converges to a Gaussian and we recover S-SNE. When $\lambda=1$, this is regular t-SNE. The most interesting region is $\lambda\in (0, 1/2)$, which corresponds to a theoretical t-distribution with $-1$ to $0$ degrees of freedom. This allows for extremely heavy tails, which in turn reveals within-cluster structures not visible with standard t-SNE. For example, [3] show $\lambda=0.5$ splits MNIST clusters into groupings based on writing style: one sees $5$ groups of $1$s based on the slant, as well as several groups of $4$s based on whether or not the top portion is closed. We do not have the computational resources to reproduce their experiments, but nonetheless wanted to implement this generalization.

## 3. Optimization

### 3.1. Opportunities for Improvement

Profiling our initial Python implementation revealed three main bottlenecks: optimizng $\sigma_i$ to the target perplexity, computing pairwise distances between the $y_i$, and computing the gradient. Pairwise distances can be computed efficiently with broadcasting; ptimizing variances and computing the gradient can be done in parallel. Our main function, `tsne`, accepts an argument `optim` which can take values in {`"none"`, `"fast"`, `"fastest"`} that correspond to combinations of these improvements (as well as other, less impactful changes such as decorating with `@jit`). Key differences between these choices are listed below.

**Basic Python:**  
* Pairwise distances are computed in a double `for` loop
* Gradient is computed one entry at a time in a triple `for` loop.

**Vectorization and Scikit-learn:**

* Pairwise distances use sklearn's `pairwise` function.
* Each row of the gradient is written as a broadcasted dot product with numpy.

**Numba and Parallelization:**

* Same `PCA`and `pairwise` functions as above.
* The $\sigma_i$ are optimized in parallel.
* Rows of the gradient are computed in parallel.

We have used `np.allclose` to verify that different levelsof optimization do indeed produce the same outputs.

### 3.2. Benchmarking
We first test each version of the three critical functions on simualted data. The following simulations are performed with $1000$ points of ambient data drawn from a $20$-dimensional standard normal and $1000$ points of embedded data drawn from a $3$ dimensional standard normal. The difference $p_{ij}-q_{ij}$ is drawn from a Gamma($1,1$) distribution. None of these distributional choices significantly influence the results; only the dimensions matter.

First, we study the distance function.

In [ ]:
from tsne import *

X = np.random.normal(0, 1, [1000, 20])
D = get_dists(X)
R = np.random.gamma(1, 1, [1000, 1000])
Y = np.random.normal(0, 1, [1000, 3])
Y_dists = get_Y_dists(Y)

In [ ]:
for optim in ["none", "fastest"]:
    %timeit get_dists(X, optim)

Moving from a double for loop to the optimized sklearn function results in <font color=red>**###**</font>-fold speed-up. This is a tremendous improvement, since the distance function is run at every iteration of t-SNE. Next we consider the effects of optimizing variances in parallel.

In [ ]:
for optim in ["none", "fastest"]:
    %timeit get_P(D, optim = optim)

Parallel computation produces <font color=red>**###**</font>-fold speed-up. The gains from this are only marginal, as we only need to optimize variances once upon initialization. Finally, we study the effects of vectorizing the gradient as well as computing rows in parallel.

In [ ]:
for optim in ["none", "fast", "fastest"]:
    %timeit get_grad(R, Y_dists, Y, optim)

Vectorization produces <font color=red>**###**</font>-fold speed-up and parallelization produces <font color=red>**###**</font>-fold speed-up. The gradient is computed at every iteration, so this is a major improvement. 

For completeness, we also run the full algorithm at all three levels of optimization. Note the slowest version of the algorithm is run for **only** 50 iterations, whereas the two fastest versions are run for $1000$ iterations. There are two reasons for this choice. First, the slowest version takes an hour for $1000$ iterations (hence $7$ hours to benchmark). Second, the gains from parallel computing are not visible for a small number of iterations due to the additional overhead of starting parallel processes.

In [ ]:
%timeit tsne(X = X, niter = 50, optim = "none", verbose = False) 
%timeit tsne(X = X, niter = 1000, optim = "fast", verbose = False)
%timeit tsne(X = X, niter = 1000, optim = "fastest", verbose = False)

Vectorization produces <font color=red>**###**</font>-fold speed-up and parallelization produces <font color=red>**###**</font>-fold speed-up. A raw Python implementation of t-SNE would not be useful for analyzing large modern datasets - parallelization is required.

## Testing

**To be done.**

## Applications 

### Simulations

### Real Data and Comparisons

**TBD. Need to write nice custom scripts for generating test cases. Should be its own `.py` file as well.**

## Discussion
**TBD**

## Contributions

**Before starting we all implemented t-SNE from scratch to make sure we understood the algorithm. We were all off by the same sign. There is no clear division or attribution for the main body of code; we pooled the best functions from our initial versions.**


Steven - Background, Description of algorithm, Optimization/Benchmarking (refactoring code + writing), testing.  
Rob - Simulations, package publishing, github organization, testing, editing, references.  
Marc - Testing (writeup + code), real data, competing algorithms, editing,  

## References
**TBD**